In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import re
pd.set_option('display.max_columns', 999)

#### Connect to SQLite database

In [9]:
db = sqlite3.connect('data/NPPES_Data_Dissemination_February_2021/npidata2.sqlite')

#### SQL query for npi data

In [ ]:
query = """
        SELECT * 
        FROM npidata
        LIMIT 5
 """

#### Assign query to variable

In [ ]:
npi_sqlite = pd.read_sql(query, db) 

#### View npi data variable

In [ ]:
npi_sqlite

#### Run query to join npi data, CBSA data and Taxonomy classification with filter on General Acute Care Hospitals, entity type 2 and CBSA

In [ ]:
query1 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hops AS h 
        on n.NPI = h.to_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE Entity_Type_Code = 2.0 AND Classification = 'General Acute Care Hospital'AND CBSA = 34980  
 """

#### Assign variable to above query 

In [ ]:
npi_sqlite1 = pd.read_sql(query1, db)

#### View query

In [ ]:
npi_sqlite1

#### Run query to join npi data, CBSA data and Taxonomy classification with filter on entity type 1 and CBSA

In [ ]:
query2 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hops AS h 
        on n.NPI = h.from_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE Entity_Type_Code = 1.0 AND CBSA = 34980  
 """

#### Assign variable to above query 

In [ ]:
from npi = type code 1, to npi = type code 2

#### View query above

In [ ]:
npi_sqlite2

In [ ]:
query3 = """
 SELECT h.from_npi,h2.to_npi,Entity_Type_Code
        FROM npidata AS n
        INNER JOIN hops AS h 
        on n.NPI = h.from_npi
        INNER JOIN hops AS h2 
        on n.NPI = h2.to_npi
"""

In [ ]:
npi_sqlite3 = pd.read_sql(query3, db)

In [ ]:
npi_sqlite3

In [27]:
query4 = """
    WITH npi_entity_type_1 AS (
        SELECT NPI AS Phys_NPI,
        n.Provider_Organization_Name AS Phys_Office,
        n.Provider_Last_Name AS Phys_Last_Name, 
        n.Provider_First_Name AS Phys_First_Name, 
        t.Classification AS Phys_Class, 
        t.Specialization AS Phys_Spec
        FROM npidata as n
        INNER JOIN CBSA as c
        ON n.Address_Postal_Codes = c.ZIP
        INNER JOIN taxonomy as t
        ON n.Healthcare_Provider_Taxonomy_Code = t.Code
        WHERE Entity_Type_Code = 1.0 AND CBSA = 34980
    ), npi_entity_type_2 AS (
        SELECT NPI AS Hosp_NPI, 
        n.Provider_Organization_Name AS Hospital,
        t.Classification AS Hosp_Class, 
        t.Specialization AS Hosp_Spec
        FROM npidata as n
        INNER JOIN CBSA as c
        ON n.Address_Postal_Codes = c.ZIP
        INNER JOIN taxonomy as t
        ON n.Healthcare_Provider_Taxonomy_Code = t.Code
        WHERE Entity_Type_Code = 2.0 AND CBSA = 34980 AND Classification = 'General Acute Care Hospital'
    )
    SELECT from_npi, 
    to_npi, 
    patient_count, 
    transaction_count,
    average_day_wait, 
    Phys_NPI,
    Phys_Office,
    Phys_Last_Name, 
    Phys_First_Name, 
    Phys_Class,
    Phys_Spec,
    Hosp_NPI,
    Hospital,
    Hosp_Class,
    Hosp_Spec
    FROM hops
    INNER JOIN npi_entity_type_1
    ON hops.from_npi = npi_entity_type_1.Phys_NPI
    INNER JOIN npi_entity_type_2
    ON hops.to_npi = npi_entity_type_2.Hosp_NPI  
    WHERE from_npi IN (SELECT Phys_NPI FROM npi_entity_type_1)
    AND to_npi IN (SELECT Hosp_NPI FROM npi_entity_type_2)
"""

In [28]:
filtered_hop_teaming = pd.read_sql(query4, db)

In [32]:
filtered_hop_teaming

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,Phys_NPI,Phys_Office,Phys_Last_Name,Phys_First_Name,Phys_Class,Phys_Spec,Hosp_NPI,Hospital,Hosp_Class,Hosp_Spec
0,1003819046,1023055126,11,11,13.182,1003819046,None,NYLANDER,BARBARA,Obstetrics & Gynecology,Gynecology,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None
1,1750384780,1023055126,48,63,36.921,1750384780,None,PERRIGIN,JULIE,Family Medicine,None,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None
2,1750384780,1023055126,48,63,36.921,1750384780,None,PERRIGIN,JULIE,Family Medicine,None,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None
3,1750384780,1396882205,24,58,32.759,1750384780,None,PERRIGIN,JULIE,Family Medicine,None,1396882205,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None
4,1750384780,1396882205,24,58,32.759,1750384780,None,PERRIGIN,JULIE,Family Medicine,None,1396882205,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35726,1558887091,1164590386,17,17,7.294,1558887091,None,DUNCAN,JEFFREY,"Nurse Anesthetist, Certified Registered",None,1164590386,SAINT THOMAS RUTHERFORD HOSPITAL,General Acute Care Hospital,None
35727,1760908362,1447571658,13,15,12.200,1760908362,None,BLATT,KATHERINE,Physical Therapist,None,1447571658,SUMNER REGIONAL MEDICAL CENTER LLC,General Acute Care Hospital,None
35728,1306362637,1780778969,21,21,1.000,1306362637,None,WILGING,ERIN,"Nurse Anesthetist, Certified Registered",None,1780778969,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None
35729,1063923407,1396882205,13,13,7.231,1063923407,None,PICKENS,REBECCA,Nurse Practitioner,Acute Care,1396882205,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None


In [30]:
filtered_hop_teaming.to_csv("data/Filtered_Hop_Team_Data")

In [7]:
filtered_hop_teaming.to_sql('hop_filtered', db, if_exists = 'append', index = False)

In [8]:
db.close()

In [ ]:
query1 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hop_filtered AS h 
        on n.NPI = h.to_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE CBSA = 34980 AND Classification = 'General Acute Care Hospital'
 """

Questions to consider with the data:
- How do we combine and filter datasets to have npi_from to be entity type 1's only and to_npi to be entity type 2's only
- Once we have the completed dataset, Are there any filters we want to put on taxonomy classification types (i.e. drama therapists, veterinarian, bus, train etc.)
- What hospitals are we considering in this analysis?